<a href="https://colab.research.google.com/github/UCLM-ESI-NECULA/DataMining/blob/master/editing_for_datacard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf sample_data/


In [ ]:
DS1  = 'DS1-DatosConsumoAlimentario.txt'
DS2  = 'DS2-PreciosSemanales.txt'
DS3A = 'DS3A-DatosMercaMadrid.txt'
DS3B = 'DS3B-DatosMercaBarna.txt'
DS4  = 'DS4-ComercioExterior.txt'
DS5  = 'DS5-CoronavirusCases.txt'
DS6  = 'DS6-owid-covid-data.csv'


DS3Transformed = 'DS3-Transformed'

# Directory path
dir_path = '/content/drive/MyDrive/mycolab/'

#Accessing the dataframes

In [ ]:
df1 = pd.read_csv(dir_path + DS1, sep='|',decimal=',')
df1_products = sorted(df1['Producto'].unique())
df1.head()

df2 = pd.read_csv(dir_path + DS2, sep='|',decimal=',')
products2 = df2['PRODUCTO'].unique()
df2_products=[]

for i in products2:
  if i.strip() not in df2_products:
    df2_products.append(i.rstrip())

df2.head()

df3 = pd.read_csv(dir_path + DS3, sep='|',decimal=',')
products3 = sorted(df3['product'].unique())
df3_products =[]

for i in products3:
  if i.strip() not in df2_products:
    df3_products.append(i.rstrip())
df3.head()

print(df1_products)
print(df2_products)
print(df3_products)


In [ ]:
#comparing the products of 1 and 3 in order to understand what we don't have in common
common_13=[]

for product in df3_products:
  if product in df1_products:
    common_13.append(product)

not_common= []

for p in df1_products:
  if p not in df3_products:
    not_common.append(p)

print(df1_products)
print(not_common)


#Categorizing the fruits of vegetables of ds1

In [ ]:
#Creating a dictionary for df3 so we can use it to label the products in df1

product_types = {}

for i1, row1 in df3.iterrows():
    product_name = row1['product']
    types = row1['familia']
    product_types[product_name] = types

def categorize_product(row):
    frutas = ['FRUTAS IV GAMA','OTRAS FRUTAS FRESCAS','T.FRUTAS FRESCAS']
    hortalizas =['OTR.HORTALIZAS/VERD.','PATATAS FRESCAS', 'PATATAS CONGELADAS',
                 'PATATAS FRITAS', 'PATATAS PROCESADAS', 'T.HORTALIZAS FRESCAS',
                 'TOTAL PATATAS','VERD./HORT. IV GAMA','VERDURAS DE HOJA']
    product_name = row['Producto']
    if product_name in product_types:
        return product_types[product_name]
    elif product_name in frutas:
      product_type='FRUTAS'
      product_types[product_name]=product_type
      return product_types[product_name]
    elif product_name in hortalizas:
      product_type='HORTALIZAS'
      product_types[product_name]=product_type
      return product_types[product_name]
    else:
      return 'Other'

#Applying categorization to create a new column 'Grupo' in df1

df1['Grupo'] = df1.apply(categorize_product, axis=1)
df1.reindex()
df1 = df1.reindex(columns=['Año',	'Mes',	'CCAA',	'Producto', 'Grupo' ,'Volumen (miles de kg)',	'Valor (miles de €)','Precio medio kg',	'Consumo per capita',	'Gasto per capita'])



#Merge

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/datacard/latest_datasets/Labeled_Cleaned_DS1-DatosConsumoAlimentario.txt', sep='|')
df2 = pd.read_csv('/content/drive/MyDrive/datacard/latest_datasets/Cleaned_DS4-ComercioExterior.txt', sep='|')
df3 = pd.read_csv('/content/drive/MyDrive/datacard/latest_datasets/Cleaned_DS6-owid-covid-data.csv')


#Rename columns in df1
df1 = df1.rename(columns={'Año': 'YEAR', 'Mes': 'MONTH'})

#Rename columns in df3
df3 = df3.rename(columns={'year': 'YEAR', 'month': 'MONTH', 'total_cases': 'TOTAL_CASES'})

#Merge datasets based on 'YEAR' and 'MONTH'
merged_df = pd.merge(df1, df2, on=['YEAR', 'MONTH'], how='inner')
merged_df = pd.merge(merged_df, df3, on=['YEAR', 'MONTH'], how='left')

#Replacing the NaN values with 0
merged_df.fillna(0, inplace=True)

merged_df.head()
print(merged_df)

#Save the merged dataset to a CSV file
merged_df.to_csv('/content/drive/MyDrive/datacard/latest_datasets/unified_dataset.csv', index=False)

#Display
merged_df.head()

# Yeni Bölüm